# Darwin Mini: ZMP trajectory

Learning from Stephane Caron's introductory article: 
https://scaron.info/teaching/prototyping-a-walking-pattern-generator.html

I would like to create a simple program to determine the optimal COM trajectory such that the ZMP contraints are enforced.  If possible, I don't want to depend on `pymanoid`, because it requires OpenRAVE, which is a pain to install (2h to install on an Ubuntu 18.04 VM!)

I found the following optimizer, which seems to be fit my needs:
https://osqp.org/docs/examples/mpc.html

In [2]:
from numpy import array, zeros

In [3]:
class Point(object):

    """
    Point
    
    pos : array, shape=(3,), optional
        Initial position in the world frame.
    vel : array, shape=(3,), optional
        Initial velocity in the world frame.
    accel : array, shape=(3,), optional
        Initial acceleration in the world frame.
    """

    def __init__(self, pos=None, vel=None, accel=None):
        self.__pos = zeros(3) if pos is None else array(pos)
        self.__pd = zeros(3) if vel is None else array(vel)
        self.__pdd = zeros(3) if accel is None else array(accel)

    def copy(self):
        """
        Copy constructor.
        
        color : char, optional
            Color of the copy, in ['r', 'g', 'b'].
        visible : bool, optional
            Should the copy be visible?
        """
        return Point(self.p, self.pd)

    @property
    def p(self):
        """Position coordinates `[x y z]` in the world frame."""
        return self.__pos.copy()

    @property
    def pos(self):
        """Position coordinates `[x y z]` in the world frame."""
        return self.p

    def set_pos(self, pos):
        """
        Set the position of the body in the world frame.
        
        pos : array, shape=(3,)
            3D vector of position coordinates.
        """
        self.__pos = array(pos)

    @property
    def pd(self):
        """Point velocity."""
        return self.__pd.copy()

    def set_vel(self, pd):
        """
        Update the point velocity.
        
        pd : array, shape=(3,)
            Velocity coordinates in the world frame.
        """
        self.__pd = array(pd)

    @property
    def pdd(self):
        """Point acceleration."""
        return self.__pdd.copy()

    def set_accel(self, pdd):
        """
        Update the point acceleration.
        
        pdd : array, shape=(3,)
            Acceleration coordinates in the world frame.
        """
        self.__pdd = array(pdd)

    def integrate_constant_accel(self, pdd, dt):
        """
        Apply Euler integration for a constant acceleration.
        
        pdd : array, shape=(3,)
            Point acceleration in the world frame.
        dt : scalar
            Duration in [s].
        """
        self.set_pos(self.p + (self.pd + .5 * pdd * dt) * dt)
        self.set_vel(self.pd + pdd * dt)
        self.set_accel(pdd)

    def integrate_constant_jerk(self, pddd, dt):
        """
        Apply Euler integration for a constant jerk.
        
        pddd : array, shape=(3,)
            Point jerk in the world frame.
        dt : scalar
            Duration in [s].
        """
        self.set_pos(self.p + dt * (
            self.pd + .5 * dt * (self.pdd + dt * pddd / 3.)))
        self.set_vel(self.pd + dt * (self.pdd + dt * pddd / 2.))
        self.set_accel(self.pdd + dt * pddd)

In [4]:
def generate_footsteps(distance, step_length, foot_spread):
    footsteps = []

    def append_contact(x, y):
        L = 0.088
        w = 0.048
        footsteps.append([(x + L/2, y + w/2), (x + L/2, y - w/2), (x - L/2, y - w/2), (x - L/2, y + w/2)])

    append_contact(0., +foot_spread)
    append_contact(0., -foot_spread)
    x = 0.
    y = foot_spread
    while x < distance:
        if distance - x <= step_length:
            x += min(distance - x, 0.5 * step_length)
        else:  # still some way to go
            x += step_length
        y = -y
        append_contact(x, y)
    append_contact(x, -y)  # now x == distance
    return footsteps

footsteps = generate_footsteps(distance=1.0, step_length=0.1, foot_spread=0.032)

In [5]:
len(footsteps)

14

In [6]:
dt = 0.03  # simulation time step, in [s]
h = 0.088
com = Point([0., 0., h])

In [ ]:
class WalkingFSM(object):

    def __init__(self, ssp_duration, dsp_duration):
        self.dsp_duration = dsp_duration
        self.mpc_timestep = 3 * dt  # update MPC every 90 [ms] (see below)
        self.next_footstep = 2
        self.ssp_duration = ssp_duration
        self.state = None
        self.start_standing()

    def start_standing(self):
        self.start_walking = False
        self.state = "Standing"
        return self.run_standing()

    def run_standing(self):
        if self.start_walking:
            self.start_walking = False
            if self.next_footstep < len(footsteps):
                return self.start_double_support()

    def start_double_support(self):
        if self.next_footstep % 2 == 1:  # left foot swings
            self.stance_foot = stance.right_foot
        else:  # right foot swings
            self.stance_foot = stance.left_foot
        dsp_duration = self.dsp_duration
        if self.next_footstep == 2 or self.next_footstep == len(footsteps) - 1:
            # double support is a bit longer for the first and last steps
            dsp_duration = 4 * self.dsp_duration
        self.swing_target = footsteps[self.next_footstep]
        self.rem_time = dsp_duration
        self.state = "DoubleSupport"
        self.start_com_mpc_dsp()
        return self.run_double_support()

    def run_double_support(self):
        if self.rem_time <= 0.:
            return self.start_single_support()
        self.run_com_mpc()
        self.rem_time -= dt

    def start_com_mpc_dsp(self):
        self.update_mpc(self.rem_time, self.ssp_duration)

    def run_com_mpc(self):
        stance.com.set_x(0.5 * (self.swing_foot.x + self.stance_foot.x))

    def start_single_support(self):
        if self.next_footstep % 2 == 1:  # left foot swings
            self.swing_foot = stance.free_contact('left_foot')
        else:  # right foot swings
            self.swing_foot = stance.free_contact('right_foot')
        self.next_footstep += 1
        self.rem_time = self.ssp_duration
        self.state = "SingleSupport"
        self.start_swing_foot()
        self.start_com_mpc_ssp()
        self.run_single_support()

    def run_single_support(self):
        if self.rem_time <= 0.:
            stance.set_contact(self.swing_foot)
            if self.next_footstep < len(footsteps):
                return self.start_double_support()
            else:  # footstep sequence is over
                return self.start_standing()
        self.run_swing_foot()
        self.run_com_mpc()
        self.rem_time -= dt

    def start_swing_foot(self):
        self.swing_start = self.swing_foot.pose
        self.swing_interp = SwingFoot(
            self.swing_foot, self.swing_target, ssp_duration,
            takeoff_clearance=0.05, landing_clearance=0.05)

    def start_com_mpc_ssp(self):
        self.update_mpc(0., self.rem_time)

    def run_swing_foot(self):
        new_pose = self.swing_interp.integrate(dt)
        self.swing_foot.set_pose(new_pose)

    def on_tick(self, sim):
        if self.state == "Standing":
            return self.run_standing()
        elif self.state == "DoubleSupport":
            return self.run_double_support()
        elif self.state == "SingleSupport":
            return self.run_single_support()
        raise Exception("Unknown state: " + self.state)

    def update_mpc(self, dsp_duration, ssp_duration):
        nb_preview_steps = 16
        T = self.mpc_timestep
        nb_init_dsp_steps = int(round(dsp_duration / T))
        nb_init_ssp_steps = int(round(ssp_duration / T))
        nb_dsp_steps = int(round(self.dsp_duration / T))
        A = array([[1., T, T ** 2 / 2.], [0., 1., T], [0., 0., 1.]])
        B = array([T ** 3 / 6., T ** 2 / 2., T]).reshape((3, 1))
        h = com.p[2]  # z
        g = -9.8  # m/s2
        zmp_from_state = array([1., 0., -h / g])
        C = array([+zmp_from_state, -zmp_from_state])
        D = None
        e = [[], []]
        cur_vertices = self.stance_foot.get_scaled_contact_area(0.8)
        next_vertices = self.swing_target.get_scaled_contact_area(0.8)
        for coord in [0, 1]:
            cur_max = max(v[coord] for v in cur_vertices)
            cur_min = min(v[coord] for v in cur_vertices)
            next_max = max(v[coord] for v in next_vertices)
            next_min = min(v[coord] for v in next_vertices)
            e[coord] = [
                array([+1000., +1000.]) if i < nb_init_dsp_steps else
                array([+cur_max, -cur_min]) if i - nb_init_dsp_steps <= nb_init_ssp_steps else
                array([+1000., +1000.]) if i - nb_init_dsp_steps - nb_init_ssp_steps < nb_dsp_steps else
                array([+next_max, -next_min])
                for i in range(nb_preview_steps)]
        self.x_mpc = LinearPredictiveControl(
            A, B, C, D, e[0],
            x_init=array([stance.com.x, stance.com.xd, stance.com.xdd]),
            x_goal=array([self.swing_target.x, 0., 0.]),
            nb_steps=nb_preview_steps,
            wxt=1., wu=0.01)
        self.y_mpc = LinearPredictiveControl(
            A, B, C, D, e[1],
            x_init=array([stance.com.y, stance.com.yd, stance.com.ydd]),
            x_goal=array([self.swing_target.y, 0., 0.]),
            nb_steps=nb_preview_steps,
            wxt=1., wu=0.01)
        self.x_mpc.solve()
        self.y_mpc.solve()
        self.preview_time = 0.

    def run_com_mpc(self):
        if self.preview_time >= self.mpc_timestep:
            if self.state == "DoubleSupport":
                self.update_mpc(self.rem_time, self.ssp_duration)
            else:  # self.state == "SingleSupport":
                self.update_mpc(0., self.rem_time)
        com_jerk = array([self.x_mpc.U[0][0], self.y_mpc.U[0][0], 0.])
        stance.com.integrate_constant_jerk(com_jerk, dt)
        self.preview_time += dt


In [1]:
from darwin_mini.trajectory import FootstepTrajectory

In [2]:
traj = FootstepTrajectory(0.1, .005, .05, 0., .088, 'l', 1.)

In [3]:
t1 = [traj(t) for t in [.0, .25, .5, .75, 1.]]

Positions at time 0.000:
  pelvis: (0.025, 0.000, 0.084)
  l_foot: (0.000, 0.320, 0.000)
  r_foot: (0.050, -0.320, 0.000)
stable foot vs pelvis:  dx=-0.0250, dy=0.0000, dz=0.0336
swing foot vs pelvis:   dx=0.0250, dy=0.0000, dz=0.0336, z_foot=0.0000
Positions at time 0.250:
  pelvis: (0.033, 0.000, 0.086)
  l_foot: (0.016, 0.320, 0.004)
  r_foot: (0.050, -0.320, 0.000)
stable foot vs pelvis:  dx=-0.0172, dy=0.0000, dz=0.0317
swing foot vs pelvis:   dx=0.0172, dy=0.0000, dz=0.0279, z_foot=0.0037
Positions at time 0.500:
  pelvis: (0.050, 0.000, 0.088)
  l_foot: (0.050, 0.320, 0.005)
  r_foot: (0.050, -0.320, 0.000)
stable foot vs pelvis:  dx=-0.0000, dy=0.0000, dz=0.0300
swing foot vs pelvis:   dx=-0.0000, dy=0.0000, dz=0.0250, z_foot=0.0050
Positions at time 0.750:
  pelvis: (0.067, 0.000, 0.086)
  l_foot: (0.084, 0.320, 0.004)
  r_foot: (0.050, -0.320, 0.000)
stable foot vs pelvis:  dx=0.0172, dy=0.0000, dz=0.0317
swing foot vs pelvis:   dx=-0.0172, dy=0.0000, dz=0.0279, z_foot=0.0037

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [2]:
from pypot.creatures import DarwinMini

darwin = DarwinMini(simulator='dummy')
# darwin = DarwinMini(simulator='vrep')

In [22]:
from darwin_mini.walk import WalkStraight, WalkingState
from darwin_mini.point import Point
from darwin_mini.mpc import LinearPredictiveControl

import numpy as np


FOOT_LENGTH = 0.088
FOOT_WIDTH = 0.048


class ZMPWalking(WalkStraight):

    def __init__(self, robot, distance, step_duration, plot=False, frequency=50):
        WalkStraight.__init__(self, robot, distance, step_duration, frequency)
        
        self.mpc_timestep = 3 * self.dt
        self.preview_time = self.mpc_timestep  # to force MPC update at start
        self.com = Point()
        self.axes = None
        if plot:
            self.fig, self.axes = plt.subplots(1, 1)
            self.axes.set_xlabel('X')
            self.axes.set_ylabel('Y')
            self.axes.set_xlim(-0.5, 1.5)
            self.axes.set_ylim(-1., 1.)
        self.points = []

    def get_support_bounds(self):
        l, w = FOOT_LENGTH, FOOT_WIDTH
        L = (1. if self.current_step == 0 else 0.5) * self.current_step_length
        stable_foot_pos = self.pos[self.stable_side + '_foot']
        cur_min = (stable_foot_pos[0] - l/2, stable_foot_pos[1] - w/2)
        cur_max = (stable_foot_pos[0] + l/2, stable_foot_pos[1] + w/2)
        next_min = (stable_foot_pos[0] + L - l/2, -stable_foot_pos[1] - w/2)
        next_max = (stable_foot_pos[0] + L + l/2, -stable_foot_pos[1] + w/2)
        return cur_min, cur_max, next_min, next_max

    def update_mpc(self, dsp_duration, ssp_duration):
        print("update_mpc(): dsp_duration={}, ssp_duration={}".format(dsp_duration, ssp_duration))
        nb_preview_steps = 16
        T = self.mpc_timestep
        nb_init_dsp_steps = int(round(dsp_duration / T))
        nb_init_ssp_steps = int(round(ssp_duration / T))
        nb_dsp_steps = int(round(self.dsp_duration / T))
        A = np.array([[1., T, T ** 2 / 2.], [0., 1., T], [0., 0., 1.]])
        B = np.array([T ** 3 / 6., T ** 2 / 2., T]).reshape((3, 1))
        h = self.com.p[2]  # z
        g = 9.80665  # m/s2 (ISO 80000-3)
        zmp_from_state = np.array([1., 0., -h / g])
        C = np.array([zmp_from_state, -zmp_from_state])
        D = None
        e = [[], []]
        cur_min, cur_max, next_min, next_max = self.get_support_bounds()
        # print("  update_mpc(): cur_min={}, cur_max={}, next_min={}, next_max={}".format(
        #     cur_min, cur_max, next_min, next_max))
        for k in [0, 1]:
            e[k] = [
                np.array([1000., 1000.]) if i < nb_init_dsp_steps else
                np.array([cur_max[k], -cur_min[k]]) if i - nb_init_dsp_steps <= nb_init_ssp_steps else
                np.array([1000., 1000.]) if i - nb_init_dsp_steps - nb_init_ssp_steps < nb_dsp_steps else
                np.array([next_max[k], -next_min[k]])
                for i in range(nb_preview_steps)]
        print("  update_mpc(): e = {}".format(e))
        stable_foot_pos = self.pos[self.stable_side + '_foot']
        x_target = stable_foot_pos[0] + 0.5 * self.current_step_length
        y_target = -.5 * stable_foot_pos[1]
        pos_target = (x_target, y_target, 0.)  # dummy z
        print("update_mpc(): com=({:.3f},{:.3f}), target=({:.3f}, {:.3f})".format(
            self.com.p[0], self.com.p[1], pos_target[0], pos_target[1]))
        self.x_mpc = LinearPredictiveControl(
            A, B, C, D, e[0],
            x_init=np.array([self.com.p[0], self.com.pd[0], self.com.pdd[0]]),
            x_goal=np.array([pos_target[0], 0., 0.]),
            nb_steps=nb_preview_steps,
            wxt=1., wu=0.01)
        self.y_mpc = LinearPredictiveControl(
            A, B, C, D, e[1],
            x_init=np.array([self.com.p[1], self.com.pd[1], self.com.pdd[1]]),
            x_goal=np.array([pos_target[1], 0., 0.]),
            nb_steps=nb_preview_steps,
            wxt=1., wu=0.01)
        self.x_mpc.solve()
        self.y_mpc.solve()
        self.preview_time = 0.

    def run_com_mpc(self, t):
        p = np.array(self.pos['pelvis'])
        p[1] = self.com.p[1]  # keep y
        self.com.set_pos(p)
        if self.preview_time >= self.mpc_timestep:
            if self.state == WalkingState.DOUBLE_SUPPORT:
                self.update_mpc(self.dsp_duration - t, self.ssp_duration)
            else:  # self.state == WalkingState.SINGLE_SUPPORT:
                self.update_mpc(0., self.step_duration - t)
        com_jerk = [self.x_mpc.U[0][0], self.y_mpc.U[0][0], 0.]
        print("run_com_mpc(): com_jerk = {}".format(com_jerk))
        if None not in com_jerk:
            self.com.integrate_constant_jerk(np.array(com_jerk), self.dt)
            print("run_com_mpc(): com = {}".format(self.com.p))
            self.points.append(self.com.p[:2])
        else:
            self.points.append([self.com.p[0], np.nan])
        if self.axes is not None:
            x, y = zip(*self.points)
            if self.axes.lines:
                for line in self.axes.lines:
                    line.set_xdata(x)
                    line.set_ydata(y)
            else:
                self.axes.plot(x, y)
            self.fig.canvas.draw()
                
        self.preview_time += self.dt

    def update_position(self, t):
        super().update_position(t)
        self.run_com_mpc(t)

In [25]:
w = ZMPWalking(darwin, 1.0, 1.0, plot=False)

w.start()


*** Start step 1 ***
 >> start double support phase <<
Positions at time 0.000:
  pelvis: (0.000, 0.000, 0.118)
  l_foot: (0.000, 0.320, 0.000)
  r_foot: (0.000, -0.320, 0.000)
update_mpc(): dsp_duration=0.1499639987945557, ssp_duration=0.85
  update_mpc(): e = [[array([1000., 1000.]), array([1000., 1000.]), array([0.044, 0.044]), array([0.044, 0.044]), array([0.044, 0.044]), array([0.044, 0.044]), array([0.044, 0.044]), array([0.044, 0.044]), array([0.044, 0.044]), array([0.044, 0.044]), array([0.044, 0.044]), array([0.044, 0.044]), array([0.044, 0.044]), array([0.044, 0.044]), array([0.044, 0.044]), array([0.044, 0.044])], [array([1000., 1000.]), array([1000., 1000.]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.2

OSQP exited with status 'primal infeasible'
run_com_mpc(): com_jerk = [0.004449729186478251, None, 0.0]
Positions at time 0.425:
  pelvis: (0.012, 0.000, 0.097)
  l_foot: (0.000, 0.320, 0.000)
  r_foot: (0.025, -0.320, 0.010)
run_com_mpc(): com_jerk = [0.004449729186478251, None, 0.0]
Positions at time 0.447:
  pelvis: (0.013, 0.000, 0.097)
  l_foot: (0.000, 0.320, 0.000)
  r_foot: (0.027, -0.320, 0.010)
run_com_mpc(): com_jerk = [0.004449729186478251, None, 0.0]
Positions at time 0.471:
  pelvis: (0.015, 0.000, 0.097)
  l_foot: (0.000, 0.320, 0.000)
  r_foot: (0.029, -0.320, 0.010)
update_mpc(): dsp_duration=0.0, ssp_duration=0.5294821262359619
  update_mpc(): e = [[array([0.044, 0.044]), array([0.044, 0.044]), array([0.044, 0.044]), array([0.044, 0.044]), array([0.044, 0.044]), array([0.044, 0.044]), array([0.044, 0.044]), array([0.044, 0.044]), array([0.044, 0.044]), array([0.044, 0.044]), array([1000., 1000.]), array([1000., 1000.]), array([ 0.094, -0.006]), array([ 0.094, -0.006])

OSQP exited with status 'primal infeasible'
run_com_mpc(): com_jerk = [-0.011363976523702869, None, 0.0]
Positions at time 0.833:
  pelvis: (0.025, 0.000, 0.095)
  l_foot: (0.000, 0.320, 0.000)
  r_foot: (0.050, -0.320, 0.001)
run_com_mpc(): com_jerk = [-0.011363976523702869, None, 0.0]
Positions at time 0.854:
  pelvis: (0.025, 0.000, 0.095)
  l_foot: (0.000, 0.320, 0.000)
  r_foot: (0.050, -0.320, -0.000)
run_com_mpc(): com_jerk = [-0.011363976523702869, None, 0.0]
Positions at time 0.875:
  pelvis: (0.025, 0.000, 0.095)
  l_foot: (0.000, 0.320, 0.000)
  r_foot: (0.050, -0.320, -0.001)
update_mpc(): dsp_duration=0.0, ssp_duration=0.12488889694213867
  update_mpc(): e = [[array([0.044, 0.044]), array([0.044, 0.044]), array([0.044, 0.044]), array([1000., 1000.]), array([1000., 1000.]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), arr

  r_foot: (0.050, -0.320, 0.000)
run_com_mpc(): com_jerk = [0.037472271765868044, -131.07823785974955, 0.0]
run_com_mpc(): com = [ 0.03407466 -0.18634617  0.09668783]
Positions at time 0.252:
  pelvis: (0.036, 0.000, 0.097)
  l_foot: (0.021, 0.320, 0.008)
  r_foot: (0.050, -0.320, 0.000)
run_com_mpc(): com_jerk = [0.037472271765868044, -131.07823785974955, 0.0]
run_com_mpc(): com = [ 0.03561791 -0.20672385  0.09692927]
Positions at time 0.274:
  pelvis: (0.037, 0.000, 0.097)
  l_foot: (0.024, 0.320, 0.009)
  r_foot: (0.050, -0.320, 0.000)
update_mpc(): dsp_duration=0.0, ssp_duration=0.7260630130767822
  update_mpc(): e = [[array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([1000., 1000.]), array([1000., 1000.])

Positions at time 0.654:
  pelvis: (0.068, 0.000, 0.096)
  l_foot: (0.087, 0.320, 0.007)
  r_foot: (0.050, -0.320, 0.000)
run_com_mpc(): com_jerk = [-0.045742180479453325, -6.344499542715055, 0.0]
run_com_mpc(): com = [ 0.06851233 -0.29775296  0.09628438]
Positions at time 0.676:
  pelvis: (0.070, 0.000, 0.096)
  l_foot: (0.089, 0.320, 0.007)
  r_foot: (0.050, -0.320, 0.000)
update_mpc(): dsp_duration=0.0, ssp_duration=0.32356905937194824
  update_mpc(): e = [[array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([ 0.094, -0.006]), array([1000., 1000.]), array([1000., 1000.]), array([ 0.144, -0.056]), array([ 0.144, -0.056]), array([ 0.144, -0.056]), array([ 0.144, -0.056]), array([ 0.144, -0.056]), array([ 0.144, -0.056]), array([ 0.144, -0.056]), array([ 0.144, -0.056])], [array([-0.296,  0.344]), array([-0.296,  0.344]), array([-0.296,  0.344]), array([-0.296,  0.344]), array([-0.296,  0.344]), array([-0.29

...

run_com_mpc(): com_jerk = [0.03763886070057738, 415.2032564176366, 0.0]
run_com_mpc(): com = [ 0.92149499 -0.01610886  0.095697  ]
Positions at time 0.132:
  pelvis: (0.921, 0.000, 0.096)
  l_foot: (0.900, 0.320, 0.000)
  r_foot: (0.942, -0.320, -0.007)
run_com_mpc(): com_jerk = [0.03763886070057738, 415.2032564176366, 0.0]
run_com_mpc(): com = [ 0.92149411 -0.06729099  0.095697  ]
 >> start single support phase <<
Positions at time 0.157:
  pelvis: (0.929, 0.000, 0.096)
  l_foot: (0.909, 0.320, 0.006)
  r_foot: (0.950, -0.320, 0.000)
run_com_mpc(): com_jerk = [0.03763886070057738, 415.2032564176366, 0.0]
run_com_mpc(): com = [ 0.92984864 -0.11434876  0.09582671]
Positions at time 0.181:
  pelvis: (0.931, 0.000, 0.096)
  l_foot: (0.912, 0.320, 0.007)
  r_foot: (0.950, -0.320, 0.000)
update_mpc(): dsp_duration=0.0, ssp_duration=0.8194780349731445
  update_mpc(): e = [[array([ 0.994, -0.906]), array([ 0.994, -0.906]), array([ 0.994, -0.906]), array([ 0.994, -0.906]), array([ 0.994, -0.90

run_com_mpc(): com_jerk = [-0.017593761073712015, -8.850101595097408, 0.0]
run_com_mpc(): com = [ 0.95836642 -0.31653485  0.09767551]
Positions at time 0.538:
  pelvis: (0.960, 0.000, 0.098)
  l_foot: (0.970, 0.320, 0.009)
  r_foot: (0.950, -0.320, 0.000)
run_com_mpc(): com_jerk = [-0.017593761073712015, -8.850101595097408, 0.0]
run_com_mpc(): com = [ 0.960165   -0.31736317  0.09751227]
Positions at time 0.562:
  pelvis: (0.962, 0.000, 0.097)
  l_foot: (0.973, 0.320, 0.009)
  r_foot: (0.950, -0.320, 0.000)
run_com_mpc(): com_jerk = [-0.017593761073712015, -8.850101595097408, 0.0]
run_com_mpc(): com = [ 0.96209987 -0.31813117  0.09729928]
Positions at time 0.583:
  pelvis: (0.963, 0.000, 0.097)
  l_foot: (0.977, 0.320, 0.009)
  r_foot: (0.950, -0.320, 0.000)
update_mpc(): dsp_duration=0.0, ssp_duration=0.4167609214782715
  update_mpc(): e = [[array([ 0.994, -0.906]), array([ 0.994, -0.906]), array([ 0.994, -0.906]), array([ 0.994, -0.906]), array([ 0.994, -0.906]), array([ 0.994, -0.906

Positions at time 0.936:
  pelvis: (0.973, 0.000, 0.095)
  l_foot: (0.997, 0.320, -0.004)
  r_foot: (0.950, -0.320, 0.000)
update_mpc(): dsp_duration=0.0, ssp_duration=0.06399917602539062
  update_mpc(): e = [[array([ 0.994, -0.906]), array([ 0.994, -0.906]), array([1000., 1000.]), array([1000., 1000.]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956])], [array([-0.296,  0.344]), array([-0.296,  0.344]), array([1000., 1000.]), array([1000., 1000.]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296

  update_mpc(): e = [[array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([1000., 1000.]), array([1000., 1000.]), array([ 1.069, -0.981]), array([ 1.069, -0.981])], [array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([1000., 1000.]), array([1000., 1000.]), array([-0.296,  0.344]), array([-0.296,  0.344])]]
update_mpc(): com=(0.984,-0.158), target=(1.025, -0.160)
OSQP exited with status 'primal infeasible'
run_com_mpc(): com_jerk = [0.0019842465604753257, None, 0.0]
Positions at time 0.3

Positions at time 0.759:
  pelvis: (0.999, 0.000, 0.098)
  l_foot: (1.000, 0.320, 0.000)
  r_foot: (0.998, -0.320, 0.004)
update_mpc(): dsp_duration=0.0, ssp_duration=0.24083995819091797
  update_mpc(): e = [[array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([ 1.044, -0.956]), array([1000., 1000.]), array([1000., 1000.]), array([ 1.069, -0.981]), array([ 1.069, -0.981]), array([ 1.069, -0.981]), array([ 1.069, -0.981]), array([ 1.069, -0.981]), array([ 1.069, -0.981]), array([ 1.069, -0.981]), array([ 1.069, -0.981]), array([ 1.069, -0.981])], [array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([ 0.344, -0.296]), array([1000., 1000.]), array([1000., 1000.]), array([-0.296,  0.344]), array([-0.296,  0.344]), array([-0.296,  0.344]), array([-0.296,  0.344]), array([-0.296,  0.344]), array([-0.296,  0.344]), array([-0.296,  0.344]), array([-0.296,  0.344]), array([-0.296,  0.344]